In [1]:
%cd /kaggle/working
!git clone https://github.com/lhldanh/Autoencoder-based-unsupervised-feature-learning-system.git
%cd Autoencoder-based-unsupervised-feature-learning-system/

/kaggle/working
Cloning into 'Autoencoder-based-unsupervised-feature-learning-system'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 211 (delta 120), reused 145 (delta 64), pack-reused 0 (from 0)
Receiving objects: 100% (211/211), 116.83 KiB | 4.33 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/kaggle/working/Autoencoder-based-unsupervised-feature-learning-system


In [2]:
%mkdir -p build
%mkdir -p weights
%mkdir -p data
!wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz -O data/cifar-10-binary.tar.gz
!tar -xzvf data/cifar-10-binary.tar.gz -C data

--2025-12-13 15:49:46--  https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170052171 (162M) [application/x-gzip]
Saving to: ‘data/cifar-10-binary.tar.gz’

data/cifar-10-binar 100%[===================>] 162.17M  59.6MB/s    in 2.7s    

2025-12-13 15:49:48 (59.6 MB/s) - ‘data/cifar-10-binary.tar.gz’ saved [170052171/170052171]

cifar-10-batches-bin/
cifar-10-batches-bin/data_batch_1.bin
cifar-10-batches-bin/batches.meta.txt
cifar-10-batches-bin/data_batch_3.bin
cifar-10-batches-bin/data_batch_4.bin
cifar-10-batches-bin/test_batch.bin
cifar-10-batches-bin/readme.html
cifar-10-batches-bin/data_batch_5.bin
cifar-10-batches-bin/data_batch_2.bin


In [3]:
!ls

build  data  include  README.md  src  weights


In [4]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


## phase 4

In [7]:
!ls weights

In [8]:
%cp -r '/kaggle/input/ltss-weights-phase3/Autoencoder-based-unsupervised-feature-learning-system/weights' ./

In [9]:
!ls weights

dec_b3.bin  dec_w4.bin	enc_w1.bin	opt_dec_b5.bin	opt_enc_b1.bin
dec_b4.bin  dec_w5.bin	enc_w2.bin	opt_dec_w3.bin	opt_enc_b2.bin
dec_b5.bin  enc_b1.bin	opt_dec_b3.bin	opt_dec_w4.bin	opt_enc_w1.bin
dec_w3.bin  enc_b2.bin	opt_dec_b4.bin	opt_dec_w5.bin	opt_enc_w2.bin


In [10]:
!ls /kaggle/working/Autoencoder-based-unsupervised-feature-learning-system

build  data  include  README.md  src  weights


In [11]:
# Bước 1: Feature Extraction
!nvcc -arch=sm_75 \
    -o build/feature_extraction \
    src/feature_extraction.cu \
    src/cifar10_dataset.cpp \
    -I include/ \
    -std=c++11 \
    -O3

%cd build/
!./feature_extraction
%cd ..

/kaggle/working/Autoencoder-based-unsupervised-feature-learning-system/build

FEATURE EXTRACTION (GPU)

Loading trained weights...
✓ Loaded weights:
  w1: 6912, b1: 256
  w2: 294912, b2: 128

Loading CIFAR-10 dataset...
--- Loading CIFAR-10 Dataset ---
Loaded batch: ../data/cifar-10-batches-bin/data_batch_1.bin | Current Total: 10000
Loaded batch: ../data/cifar-10-batches-bin/data_batch_2.bin | Current Total: 20000
Loaded batch: ../data/cifar-10-batches-bin/data_batch_3.bin | Current Total: 30000
Loaded batch: ../data/cifar-10-batches-bin/data_batch_4.bin | Current Total: 40000
Loaded batch: ../data/cifar-10-batches-bin/data_batch_5.bin | Current Total: 50000
Loaded batch: ../data/cifar-10-batches-bin/test_batch.bin | Current Total: 10000
Successfully loaded 50000 train images and 10000 test images.
✓ Loaded dataset:
  Train images: 50000
  Test images: 10000
  Using 50000 train images and 10000 test images

GPU FEATURE EXTRACTION

Processing training images...
  Processed 8000/50000 t

In [12]:
!ls /kaggle/working/Autoencoder-based-unsupervised-feature-learning-system/features

test_features.bin  train_features.bin


## read bin file for svm

In [13]:
import numpy as np
import struct
import os

def load_features_and_labels_python(filename):
    """
    Loads features and labels from a binary file saved using the specific C++ format:
    [uint32_t N] [uint32_t D] [N * D * float features] [N * int labels]

    Args:
        filename (str): Path to the binary feature/label file.

    Returns:
        tuple: (features_array (np.ndarray), labels_array (np.ndarray))
    """
    
    try:
        file_size_bytes = os.path.getsize(filename)
    except FileNotFoundError:
        print(f"Error: File not found at {filename}")
        return None, None

    with open(filename, 'rb') as file:

        header_bytes = file.read(8)
        


        try:
            num_samples, feature_dim = struct.unpack('<II', header_bytes)
        except struct.error:
            print("Error: Could not unpack 8-byte header. File might be corrupted or empty.")
            return None, None
    
    print(f"Loading {num_samples} samples with {feature_dim} features...")
    
    
    FEATURE_DATA_SIZE = num_samples * feature_dim * np.dtype(np.float32).itemsize 
    
    LABEL_DATA_SIZE = num_samples * np.dtype(np.int32).itemsize 
    
    EXPECTED_FILE_SIZE = 8 + FEATURE_DATA_SIZE + LABEL_DATA_SIZE
    
    if file_size_bytes != EXPECTED_FILE_SIZE:
        print(f"WARNING: File size mismatch!")
        print(f"  Expected: {EXPECTED_FILE_SIZE} bytes")
        print(f"  Actual:   {file_size_bytes} bytes")

        
    
    FEATURES_OFFSET = 8 
    
    features_1d = np.fromfile(
        filename, 
        dtype=np.float32, 
        offset=FEATURES_OFFSET, 
        count=num_samples * feature_dim
    )
    
    features_array = features_1d.reshape((num_samples, feature_dim))
    
    
    LABELS_OFFSET = FEATURES_OFFSET + FEATURE_DATA_SIZE 
    
    labels_array = np.fromfile(
        filename, 
        dtype=np.int32,
        offset=LABELS_OFFSET, 
        count=num_samples
    )
    
    print(f"✓ Loaded {num_samples} samples successfully.")
    print(f"Features Shape: {features_array.shape}, Labels Shape: {labels_array.shape}")
    
    return features_array, labels_array

feature_train, label_train = load_features_and_labels_python('./features/train_features.bin')
feature_test, label_test = load_features_and_labels_python('./features/test_features.bin')

Loading 50000 samples with 8192 features...
✓ Loaded 50000 samples successfully.
Features Shape: (50000, 8192), Labels Shape: (50000,)
Loading 10000 samples with 8192 features...
✓ Loaded 10000 samples successfully.
Features Shape: (10000, 8192), Labels Shape: (10000,)


In [14]:
import time
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import numpy as np
import os

# --- 0. Dummy Data Setup (Replace with your actual data loading) ---
# Assuming feature_train, label_train, feature_test, label_test are loaded np arrays
# For demonstration, we create dummy arrays:
N_TRAIN = 50000
N_TEST = 10000
N_FEATURES = 8192
N_CLASSES = 10

# NOTE: REMOVE THESE DUMMY LINES WHEN YOU RUN WITH YOUR ACTUAL DATA
# X_train = np.random.rand(N_TRAIN, N_FEATURES).astype(np.float32)
# y_train = np.random.randint(0, N_CLASSES, N_TRAIN)
# X_test = np.random.rand(N_TEST, N_FEATURES).astype(np.float32)
# y_test = np.random.randint(0, N_CLASSES, N_TEST)

# --- 1. Load Data (Use your actual variable names) ---
X_train = feature_train
y_train = label_train
X_test = feature_test
y_test = label_test


# ====================================================================
# FUNCTION TO SAVE MODEL WEIGHTS
# ====================================================================

def save_svm_weights(model, filename="linear_svm_weights.npz"):
    """
    Saves the coefficients (weights) and intercept (bias) of a trained 
    LinearSVC model to a compressed NumPy .npz file.
    """
    
    # Extract the parameters
    weights = model.coef_    # Shape: (N_CLASSES, N_FEATURES)
    bias = model.intercept_  # Shape: (N_CLASSES,)
    
    # Save them to a compressed .npz file
    # The keys 'W' and 'B' are used to retrieve the data later.
    np.savez_compressed(
        filename, 
        W=weights, 
        B=bias
    )
    
    print(f"\n--- Model Weights Saved ---")
    print(f"Weights and bias saved to: {filename}")
    print(f"Weights (W) shape: {weights.shape}")
    print(f"Bias (B) shape: {bias.shape}")
    
    return filename

# ====================================================================
# TRAINING AND EVALUATION
# ====================================================================

print("--- Starting LinearSVC Training (Optimized for Speed) ---")

svm_model = LinearSVC(
    dual=False,          # Faster when N > D (50000 > 8192)
    C=0.1,               # Lower C = faster training (more regularization)
    random_state=42,     # For reproducibility
    max_iter=2000,         # You may need to increase this if convergence fails
    tol=1e-4,            # Tolerance for stopping criteria
    verbose=1            # Enables iteration/epoch progress logging
)

start_time = time.time()

# Train the model
svm_model.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time

print(f"\nTraining Time: {training_time:.2f} seconds")

# --- 3. Evaluation ---

# Predict on the test set
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Test Set Accuracy: {accuracy * 100:.2f}%")
print(f"--- LinearSVC Optimization Complete ---")

# ====================================================================
# 4. SAVE WEIGHTS AND BIAS
# ====================================================================

save_path = save_svm_weights(svm_model, filename="final_cifar10_svm_weights.npz")

# --- Example of Loading Weights Later ---
# loaded_data = np.load(save_path)
# loaded_weights = loaded_data['W']
# loaded_bias = loaded_data['B']

--- Starting LinearSVC Training (Optimized for Speed) ---
[LibLinear]iter  1 act 2.889e+03 pre 2.887e+03 delta 9.658e-03 f 5.000e+03 |g| 5.982e+05 CG   1
cg reaches trust region boundary
iter  2 act 1.020e+02 pre 1.011e+02 delta 1.707e-02 f 2.111e+03 |g| 1.484e+04 CG   2
cg reaches trust region boundary
iter  3 act 1.595e+02 pre 1.615e+02 delta 2.723e-02 f 2.009e+03 |g| 4.819e+04 CG   2
cg reaches trust region boundary
iter  4 act 8.661e+01 pre 7.990e+01 delta 3.441e-02 f 1.850e+03 |g| 9.512e+03 CG   3
cg reaches trust region boundary
iter  5 act 3.972e+01 pre 3.848e+01 delta 4.938e-02 f 1.763e+03 |g| 7.330e+03 CG   2
cg reaches trust region boundary
iter  6 act 5.017e+01 pre 5.233e+01 delta 6.792e-02 f 1.724e+03 |g| 3.485e+03 CG   3
cg reaches trust region boundary
iter  7 act 4.890e+01 pre 4.528e+01 delta 8.316e-02 f 1.673e+03 |g| 4.368e+03 CG   3
cg reaches trust region boundary
iter  8 act 5.624e+01 pre 5.548e+01 delta 1.247e-01 f 1.624e+03 |g| 5.882e+03 CG   4
cg reaches trust reg